In [7]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from time import gmtime, strftime, localtime
from matplotlib.ticker import MultipleLocator
from scipy.stats import spearmanr
from scipy.stats import pearsonr

params = {"xtick.direction": "in", "ytick.direction": "in"}
plt.rcParams.update(params)
plt.rcParams["font.family"] = "Tahoma"

In [8]:
mhws_df = pd.read_csv('mhws_v202504.csv') # change filename for mcs, download from zenodo 10.5281/zenodo.17577678

In [9]:
oni = pd.read_csv("oni.csv")
oni["Date"] = pd.to_datetime(oni["Date"])

threshold = 0.5
min_months = 5

el_mask = oni["ONI"] >= threshold
la_mask = oni["ONI"] <= -threshold

def summarize_runs(dates, mask, label):
    starts, ends = [], []
    i, n = 0, len(mask)
    while i < n:
        if mask[i]:
            j = i
            while j < n and mask[j]:
                j += 1
            if (j - i) >= min_months:
                starts.append(dates.iloc[i])
                ends.append(dates.iloc[j-1])
            i = j
        else:
            i += 1
    return pd.DataFrame({
        "ENSO_event": label,
        "Start_date": starts,
        "End_date": ends
    })

el_events = summarize_runs(oni["Date"], el_mask.to_numpy(), "El Niño")
la_events = summarize_runs(oni["Date"], la_mask.to_numpy(), "La Niña")

enso_events = pd.concat([el_events, la_events], ignore_index=True).sort_values("Start_date").reset_index(drop=True)
enso_events

,ENSO_event,Start_date,End_date
0,La Niña,1950-01-01,1950-07-01
1,El Niño,1951-06-01,1952-01-01
2,El Niño,1953-02-01,1954-01-01
3,La Niña,1954-05-01,1956-08-01
4,El Niño,1957-04-01,1958-07-01
5,El Niño,1958-11-01,1959-03-01
6,El Niño,1963-06-01,1964-02-01
7,La Niña,1964-05-01,1965-01-01
8,El Niño,1965-06-01,1966-04-01
9,El Niño,1968-10-01,1969-05-01


In [10]:
enso_monthly = []
for _, row in enso_events.iterrows():
    months = pd.period_range(row.Start_date, row.End_date, freq="M")
    enso_monthly.extend([(m.to_timestamp(), row.ENSO_event) for m in months])
enso_monthly = pd.DataFrame(enso_monthly, columns=["month", "ENSO_event"])

full_range = pd.date_range(enso_monthly.month.min(), enso_monthly.month.max(), freq="MS")
enso_monthly = enso_monthly.set_index("month").reindex(full_range).fillna("Neutral").rename_axis("month").reset_index()

# numeric code for correlation
enso_map = {"El Niño": 1, "La Niña": -1, "Neutral": 0}
enso_monthly["ENSO_code"] = enso_monthly["ENSO_event"].map(enso_map)

mhws_df["month"] = pd.to_datetime(mhws_df["Start_date"]).dt.to_period("M").dt.to_timestamp()

eco_sites = mhws_df.groupby("Ecoregion")["Site_ID"].nunique().rename("n_sites")
eco_monthly = mhws_df.groupby(["Ecoregion", "month"]).size().rename("count").reset_index()
eco_monthly = eco_monthly.merge(eco_sites, on="Ecoregion")
eco_monthly["MHW_rate"] = eco_monthly["count"] / eco_monthly["n_sites"]

hab_sites = mhws_df.groupby("Habitat")["Site_ID"].nunique().rename("n_sites")
hab_monthly = mhws_df.groupby(["Habitat", "month"]).size().rename("count").reset_index()
hab_monthly = hab_monthly.merge(hab_sites, on="Habitat")
hab_monthly["MHW_rate"] = hab_monthly["count"] / hab_monthly["n_sites"]

eco_monthly = eco_monthly.merge(enso_monthly[["month", "ENSO_code"]], on="month", how="left")
hab_monthly = hab_monthly.merge(enso_monthly[["month", "ENSO_code"]], on="month", how="left")

def monthly_corr(df, group_col):
    results = []
    for g, sub in df.groupby(group_col):
        sub = sub.dropna(subset=["MHW_rate", "ENSO_code"])
        if len(sub) > 2 and sub["MHW_rate"].nunique() > 1:
            r, p = pearsonr(sub["MHW_rate"], sub["ENSO_code"])
            results.append((g, r, p))
        else:
            results.append((g, np.nan, np.nan))
    return pd.DataFrame(results, columns=[group_col, "r_value", "p_value"])

eco_corr = monthly_corr(eco_monthly, "Ecoregion")
hab_corr = monthly_corr(hab_monthly, "Habitat")

print("\nCorrelation per Ecoregion (MHW frequency vs ENSO phase):")
print(eco_corr.sort_values("r_value", ascending=False))

print("\nCorrelation per Habitat (MHW frequency vs ENSO phase):")
print(hab_corr.sort_values("r_value", ascending=False))


Correlation per Ecoregion (MHW frequency vs ENSO phase):
                                Ecoregion   r_value       p_value
1                          Eastern Brazil  0.262120  2.020298e-07
3                     Northeastern Brazil  0.203711  9.697614e-05
4                              Rio Grande  0.070709  3.428614e-01
6                     Southeastern Brazil  0.050996  3.138841e-01
7         Trindade and Martim Vaz Islands -0.017250  8.535407e-01
5         Sao Pedro and Sao Paulo Islands -0.028509  7.717419e-01
0                                  Amazon -0.108523  5.834725e-02
2  Fernando de Noronha and Atol das Rocas -0.155050  1.491777e-01

Correlation per Habitat (MHW frequency vs ENSO phase):
           Habitat   r_value   p_value
0    Bryozoan reef  0.275471  0.000056
1       Coral reef  0.233129  0.000005
3            Kelps  0.197062  0.000637
2    Halimeda bank  0.172223  0.003599
5    Rhodolith bed  0.159715  0.001111
7  Seagrass meadow  0.099998  0.040522
6      Rocky shore 